In [1]:
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

In [2]:
%load_ext autoreload
%autoreload 2
from data.api_fetcher import ApiFetcher

In [27]:
api = ApiFetcher(starting_year=2019, ending_year=2025)
df_p = api.get_dataframe('playerstats', all=True)

In [28]:
df_g = api.get_dataframe('leaguegamelog', all = True) 

In [17]:
print(df_p.columns)

Index(['season_year', 'player_id', 'player_name', 'nickname', 'team_id',
       'team_abbreviation', 'team_name', 'game_id', 'game_date', 'matchup',
       'wl', 'min', 'fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a', 'fg3_pct', 'ftm',
       'fta', 'ft_pct', 'oreb', 'dreb', 'reb', 'ast', 'tov', 'stl', 'blk',
       'blka', 'pf', 'pfd', 'pts', 'plus_minus', 'nba_fantasy_pts', 'dd2',
       'td3', 'wnba_fantasy_pts', 'gp_rank', 'w_rank', 'l_rank', 'w_pct_rank',
       'min_rank', 'fgm_rank', 'fga_rank', 'fg_pct_rank', 'fg3m_rank',
       'fg3a_rank', 'fg3_pct_rank', 'ftm_rank', 'fta_rank', 'ft_pct_rank',
       'oreb_rank', 'dreb_rank', 'reb_rank', 'ast_rank', 'tov_rank',
       'stl_rank', 'blk_rank', 'blka_rank', 'pf_rank', 'pfd_rank', 'pts_rank',
       'plus_minus_rank', 'nba_fantasy_pts_rank', 'dd2_rank', 'td3_rank',
       'wnba_fantasy_pts_rank', 'available_flag', 'min_sec', 'team_count',
       'season', 'ts_pct', 'ast_to_ratio', 'pts_per_fga', 'reb_per_min',
       'pts_ast_composite', 

In [5]:
print(df_g.columns)

Index(['game_id', 'date', 'home_team', 'away_team', 'home_season_id',
       'home_team_id', 'home_team_abbreviation', 'home_wl', 'home_min',
       'home_fgm', 'home_fga', 'home_fg_pct', 'home_fg3m', 'home_fg3a',
       'home_fg3_pct', 'home_ftm', 'home_fta', 'home_ft_pct', 'home_oreb',
       'home_dreb', 'home_reb', 'home_ast', 'home_stl', 'home_blk', 'home_tov',
       'home_pf', 'home_pts', 'home_plus_minus', 'home_video_available',
       'away_season_id', 'away_team_id', 'away_team_abbreviation', 'away_wl',
       'away_min', 'away_fgm', 'away_fga', 'away_fg_pct', 'away_fg3m',
       'away_fg3a', 'away_fg3_pct', 'away_ftm', 'away_fta', 'away_ft_pct',
       'away_oreb', 'away_dreb', 'away_reb', 'away_ast', 'away_stl',
       'away_blk', 'away_tov', 'away_pf', 'away_pts', 'away_plus_minus',
       'away_video_available'],
      dtype='object')


In [29]:
"""
Data validation - checks for errors and inconsistencies
Does NOT modify data, only reports issues
Customized for your specific column structure
"""
import pandas as pd
import numpy as np

def validate_games_dataframe(df_g):
    """
    Check games dataframe for issues
    
    Args:
        df_g: Games DataFrame (with your column structure)
        
    Returns:
        dict: Validation results with issues found
    """
    print("\n" + "="*60)
    print("🔍 VALIDATING GAMES DATAFRAME")
    print("="*60)
    
    issues = []
    warnings = []
    
    # Basic info
    print(f"\n📊 Dataset Info:")
    print(f"   Total games: {len(df_g)}")
    print(f"   Date range: {df_g['date'].min()} to {df_g['date'].max()}")
    print(f"   Unique teams: {df_g['home_team_id'].nunique()}")
    
    # Check 1: Missing values
    print(f"\n🔍 Checking for missing values...")
    critical_cols = ['game_id', 'date', 'home_team_id', 'away_team_id', 
                     'home_pts', 'away_pts']
    
    missing_found = False
    for col in critical_cols:
        if col not in df_g.columns:
            issues.append(f"Missing column: {col}")
            missing_found = True
        else:
            missing = df_g[col].isna().sum()
            if missing > 0:
                issues.append(f"{col}: {missing} missing values")
                missing_found = True
                print(f"   ❌ {col}: {missing} missing values")
    
    if not missing_found:
        print("   ✅ No missing values in critical columns")
    
    # Check 2: Duplicate games
    print(f"\n🔍 Checking for duplicates...")
    duplicates = df_g['game_id'].duplicated().sum()
    if duplicates > 0:
        issues.append(f"{duplicates} duplicate game_ids")
        print(f"   ❌ {duplicates} duplicate games found")
    else:
        print("   ✅ No duplicate games")
    
    # Check 3: Chronological order
    print(f"\n🔍 Checking chronological order...")
    if not df_g['date'].is_monotonic_increasing:
        issues.append("Games not sorted by date")
        print("   ❌ Games NOT in chronological order")
        print("   💡 Fix: df_g = df_g.sort_values('date').reset_index(drop=True)")
    else:
        print("   ✅ Games in chronological order")
    
    # Check 4: Score ranges
    print(f"\n🔍 Checking score ranges...")
    min_home = df_g['home_pts'].min()
    max_home = df_g['home_pts'].max()
    min_away = df_g['away_pts'].min()
    max_away = df_g['away_pts'].max()
    
    print(f"   Home scores: {min_home} to {max_home}")
    print(f"   Away scores: {min_away} to {max_away}")
    
    if min_home < 50 or min_away < 50:
        warnings.append(f"Unusually low scores found (min: {min(min_home, min_away)})")
        print(f"   ⚠️  Some scores < 50 points")
    
    if max_home > 200 or max_away > 200:
        warnings.append(f"Unusually high scores found (max: {max(max_home, max_away)})")
        print(f"   ⚠️  Some scores > 200 points")
    
    if len([w for w in warnings if 'score' in w.lower()]) == 0:
        print("   ✅ Score ranges look reasonable (50-200)")
    
    # Check 5: Data types
    print(f"\n🔍 Checking data types...")
    if not pd.api.types.is_datetime64_any_dtype(df_g['date']):
        warnings.append("'date' is not datetime type")
        print(f"   ⚠️  'date' is not datetime")
        print(f"   💡 Fix: df_g['date'] = pd.to_datetime(df_g['date'])")
    else:
        print(f"   ✅ 'date' is datetime")
    
    # Check 6: Season distribution
    print(f"\n🔍 Checking season distribution...")
    season_counts = df_g['home_season_id'].value_counts().sort_index()
    print(f"   Games per season:")
    for season, count in season_counts.items():
        print(f"      {season}: {count} games")
    
    # Check 7: Home/Away team IDs are different
    print(f"\n🔍 Checking team matchups...")
    same_team = (df_g['home_team_id'] == df_g['away_team_id']).sum()
    if same_team > 0:
        issues.append(f"{same_team} games where home_team_id == away_team_id")
        print(f"   ❌ {same_team} games with same home/away team")
    else:
        print(f"   ✅ No games where team plays itself")
    
    return {
        'issues': issues,
        'warnings': warnings,
        'passed': len(issues) == 0
    }


def validate_players_dataframe(df_p):
    """
    Check players dataframe for issues
    
    Args:
        df_p: Players DataFrame (with your column structure)
        
    Returns:
        dict: Validation results with issues found
    """
    print("\n" + "="*60)
    print("🔍 VALIDATING PLAYERS DATAFRAME")
    print("="*60)
    
    issues = []
    warnings = []
    
    # Basic info
    print(f"\n📊 Dataset Info:")
    print(f"   Total records: {len(df_p)}")
    print(f"   Date range: {df_p['game_date'].min()} to {df_p['game_date'].max()}")
    print(f"   Unique players: {df_p['player_id'].nunique()}")
    print(f"   Unique games: {df_p['game_id'].nunique()}")
    
    # Check 1: Missing values in critical columns
    print(f"\n🔍 Checking for missing values...")
    critical_cols = ['player_id', 'game_id', 'team_id', 'game_date', 'min', 'pts']
    
    missing_found = False
    for col in critical_cols:
        missing = df_p[col].isna().sum()
        if missing > 0:
            issues.append(f"{col}: {missing} missing values ({missing/len(df_p)*100:.1f}%)")
            missing_found = True
            print(f"   ❌ {col}: {missing} missing values")
    
    if not missing_found:
        print("   ✅ No missing values in critical columns")
    
    # Check 2: Duplicate player-games
    print(f"\n🔍 Checking for duplicates...")
    duplicates = df_p.duplicated(subset=['player_id', 'game_id']).sum()
    if duplicates > 0:
        issues.append(f"{duplicates} duplicate player-game records")
        print(f"   ❌ {duplicates} duplicate player-game records")
        print(f"   💡 Fix: df_p = df_p.drop_duplicates(subset=['player_id', 'game_id'])")
    else:
        print("   ✅ No duplicate player-game records")
    
    # Check 3: Minutes played ranges
    print(f"\n🔍 Checking minutes played...")
    min_minutes = df_p['min'].min()
    max_minutes = df_p['min'].max()
    zero_minutes = (df_p['min'] == 0).sum()
    under_1_min = (df_p['min'] < 1).sum()
    
    print(f"   Minutes range: {min_minutes} to {max_minutes}")
    print(f"   Players with 0 minutes: {zero_minutes}")
    print(f"   Players with <1 minute: {under_1_min}")
    
    if max_minutes > 60:
        over_60 = (df_p['min'] > 60).sum()
        warnings.append(f"{over_60} records with >60 minutes (overtime games)")
        print(f"   ⚠️  {over_60} records with >60 minutes (overtime)")
    
    if zero_minutes > len(df_p) * 0.1:
        warnings.append(f"Many players with 0 minutes ({zero_minutes})")
        print(f"   ⚠️  {zero_minutes} players with 0 minutes (DNP)")
    
    if under_1_min > 0 and under_1_min < len(df_p) * 0.05:
        print(f"   ✅ Most players have meaningful minutes")
    
    # Check 4: Stats consistency
    print(f"\n🔍 Checking stats consistency...")
    
    # FGM <= FGA
    fgm_greater = (df_p['fgm'] > df_p['fga']).sum()
    if fgm_greater > 0:
        issues.append(f"{fgm_greater} records where FGM > FGA (impossible)")
        print(f"   ❌ {fgm_greater} records where FGM > FGA")
    else:
        print(f"   ✅ FGM <= FGA for all records")
    
    # FG3M <= FG3A
    fg3m_greater = (df_p['fg3m'] > df_p['fg3a']).sum()
    if fg3m_greater > 0:
        issues.append(f"{fg3m_greater} records where FG3M > FG3A (impossible)")
        print(f"   ❌ {fg3m_greater} records where FG3M > FG3A")
    else:
        print(f"   ✅ FG3M <= FG3A for all records")
    
    # FTM <= FTA
    ftm_greater = (df_p['ftm'] > df_p['fta']).sum()
    if ftm_greater > 0:
        issues.append(f"{ftm_greater} records where FTM > FTA (impossible)")
        print(f"   ❌ {ftm_greater} records where FTM > FTA")
    else:
        print(f"   ✅ FTM <= FTA for all records")
    
    # FG3M <= FGM (3-pointers are subset of all field goals)
    fg3m_greater_fgm = (df_p['fg3m'] > df_p['fgm']).sum()
    if fg3m_greater_fgm > 0:
        issues.append(f"{fg3m_greater_fgm} records where FG3M > FGM (impossible)")
        print(f"   ❌ {fg3m_greater_fgm} records where FG3M > FGM")
    else:
        print(f"   ✅ FG3M <= FGM for all records")
    
    # Check 5: Negative values
    print(f"\n🔍 Checking for negative values...")
    stat_cols = ['min', 'pts', 'reb', 'ast', 'stl', 'blk', 'fgm', 'fga']
    negative_found = False
    
    for col in stat_cols:
        negatives = (df_p[col] < 0).sum()
        if negatives > 0:
            issues.append(f"{col}: {negatives} negative values")
            negative_found = True
            print(f"   ❌ {col}: {negatives} negative values")
    
    if not negative_found:
        print("   ✅ No negative values in stats")
    
    # Check 6: Chronological order
    print(f"\n🔍 Checking chronological order...")
    if not df_p['game_date'].is_monotonic_increasing:
        warnings.append("Players data not sorted by date")
        print("   ⚠️  Data NOT in chronological order")
        print("   💡 Fix: df_p = df_p.sort_values('game_date').reset_index(drop=True)")
    else:
        print("   ✅ Data in chronological order")
    
    # Check 7: Data types
    print(f"\n🔍 Checking data types...")
    if not pd.api.types.is_datetime64_any_dtype(df_p['game_date']):
        warnings.append("'game_date' is not datetime type")
        print(f"   ⚠️  'game_date' is not datetime")
        print(f"   💡 Fix: df_p['game_date'] = pd.to_datetime(df_p['game_date'])")
    else:
        print(f"   ✅ 'game_date' is datetime")
    
    return {
        'issues': issues,
        'warnings': warnings,
        'passed': len(issues) == 0
    }


def validate_consistency(df_g, df_p):
    """
    Check consistency between games and players dataframes
    
    Args:
        df_g: Games DataFrame
        df_p: Players DataFrame
        
    Returns:
        dict: Validation results
    """
    print("\n" + "="*60)
    print("🔍 VALIDATING CROSS-DATAFRAME CONSISTENCY")
    print("="*60)
    
    issues = []
    warnings = []
    
    # Check 1: Date range overlap
    print(f"\n🔍 Checking date ranges...")
    games_start = df_g['date'].min()
    games_end = df_g['date'].max()
    players_start = df_p['game_date'].min()
    players_end = df_p['game_date'].max()
    
    print(f"   Games:   {games_start} to {games_end}")
    print(f"   Players: {players_start} to {players_end}")
    
    if players_start > games_start:
        warnings.append(f"Player data starts later than games ({players_start} vs {games_start})")
        print(f"   ⚠️  Player data starts later than games")
    
    if players_end < games_end:
        warnings.append(f"Player data ends earlier than games ({players_end} vs {games_end})")
        print(f"   ⚠️  Player data ends earlier than games")
    
    if len(warnings) == 0:
        print(f"   ✅ Date ranges fully overlap")
    
    # Check 2: Game IDs consistency
    print(f"\n🔍 Checking game_id consistency...")
    games_ids = set(df_g['game_id'].astype(str).unique())
    players_ids = set(df_p['game_id'].astype(str).unique())
    
    games_only = games_ids - players_ids
    players_only = players_ids - games_ids
    overlap = games_ids & players_ids
    
    print(f"   Games with player data: {len(overlap)} / {len(games_ids)}")
    
    if len(games_only) > 0:
        pct = len(games_only) / len(games_ids) * 100
        warnings.append(f"{len(games_only)} games ({pct:.1f}%) have no player data")
        print(f"   ⚠️  {len(games_only)} games missing player data")
    
    if len(players_only) > 0:
        warnings.append(f"{len(players_only)} games in player data not in games data")
        print(f"   ⚠️  {len(players_only)} orphan games in player data")
    
    if len(games_only) == 0 and len(players_only) == 0:
        print(f"   ✅ Perfect game_id alignment")
    
    # Check 3: Score consistency (sample)
    print(f"\n🔍 Checking score consistency (sample of 10 games)...")
    sample_games = df_g.sample(min(10, len(df_g)))
    
    mismatches = 0
    for _, game in sample_games.iterrows():
        game_players = df_p[df_p['game_id'].astype(str) == str(game['game_id'])]
        
        home_players = game_players[game_players['team_id'] == game['home_team_id']]
        away_players = game_players[game_players['team_id'] == game['away_team_id']]
        
        if len(home_players) > 0:
            home_total = home_players['pts'].sum()
            if abs(home_total - game['home_pts']) > 1:
                mismatches += 1
        
        if len(away_players) > 0:
            away_total = away_players['pts'].sum()
            if abs(away_total - game['away_pts']) > 1:
                mismatches += 1
    
    if mismatches > 0:
        warnings.append(f"Score mismatches found in {mismatches}/20 checks")
        print(f"   ⚠️  {mismatches}/20 score mismatches (player sum ≠ team total)")
    else:
        print(f"   ✅ Player points sum to team totals")
    
    # Check 4: Team IDs match
    print(f"\n🔍 Checking team IDs...")
    games_teams = set(df_g['home_team_id'].unique()) | set(df_g['away_team_id'].unique())
    player_teams = set(df_p['team_id'].unique())
    
    teams_only_in_games = games_teams - player_teams
    teams_only_in_players = player_teams - games_teams
    
    if len(teams_only_in_games) > 0:
        warnings.append(f"{len(teams_only_in_games)} teams in games but not in player data")
        print(f"   ⚠️  {len(teams_only_in_games)} teams in games but not players")
    
    if len(teams_only_in_players) > 0:
        warnings.append(f"{len(teams_only_in_players)} teams in player data but not games")
        print(f"   ⚠️  {len(teams_only_in_players)} teams in players but not games")
    
    if len(teams_only_in_games) == 0 and len(teams_only_in_players) == 0:
        print(f"   ✅ Team IDs match across dataframes")
    
    return {
        'issues': issues,
        'warnings': warnings,
        'passed': len(issues) == 0
    }


def validate_all(df_g, df_p):
    """
    Run all validation checks
    
    Args:
        df_g: Games DataFrame
        df_p: Players DataFrame
        
    Returns:
        dict: Complete validation results
    """
    print("\n" + "🔍 "*30)
    print("STARTING COMPLETE DATA VALIDATION")
    print("🔍 "*30)
    
    # Run all validations
    games_results = validate_games_dataframe(df_g)
    players_results = validate_players_dataframe(df_p)
    consistency_results = validate_consistency(df_g, df_p)
    
    # Combine results
    all_issues = (games_results['issues'] + 
                  players_results['issues'] + 
                  consistency_results['issues'])
    
    all_warnings = (games_results['warnings'] + 
                    players_results['warnings'] + 
                    consistency_results['warnings'])
    
    # Final report
    print("\n" + "="*60)
    print("📋 VALIDATION SUMMARY")
    print("="*60)
    
    if len(all_issues) == 0:
        print("✅ NO CRITICAL ISSUES FOUND")
    else:
        print(f"❌ {len(all_issues)} CRITICAL ISSUES FOUND:")
        for i, issue in enumerate(all_issues, 1):
            print(f"   {i}. {issue}")
    
    if len(all_warnings) > 0:
        print(f"\n⚠️  {len(all_warnings)} WARNINGS:")
        for i, warning in enumerate(all_warnings, 1):
            print(f"   {i}. {warning}")
    else:
        print("\n✅ NO WARNINGS")
    
    print("\n" + "="*60)
    
    if len(all_issues) == 0 and len(all_warnings) == 0:
        print("🎉 DATA IS READY FOR MODELING!")
    elif len(all_issues) == 0:
        print("✅ DATA IS USABLE (with minor warnings)")
    else:
        print("❌ FIX CRITICAL ISSUES BEFORE PROCEEDING")
    
    print("="*60)
    
    return {
        'games': games_results,
        'players': players_results,
        'consistency': consistency_results,
        'all_issues': all_issues,
        'all_warnings': all_warnings,
        'passed': len(all_issues) == 0
    }

In [30]:
results = validate_all(df_g, df_p)


🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 
STARTING COMPLETE DATA VALIDATION
🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 🔍 

🔍 VALIDATING GAMES DATAFRAME

📊 Dataset Info:
   Total games: 7054
   Date range: 2019-10-22 to 2025-04-13
   Unique teams: 30

🔍 Checking for missing values...
   ✅ No missing values in critical columns

🔍 Checking for duplicates...
   ✅ No duplicate games

🔍 Checking chronological order...
   ✅ Games in chronological order

🔍 Checking score ranges...
   Home scores: 73 to 175
   Away scores: 67 to 176
   ✅ Score ranges look reasonable (50-200)

🔍 Checking data types...
   ⚠️  'date' is not datetime
   💡 Fix: df_g['date'] = pd.to_datetime(df_g['date'])

🔍 Checking season distribution...
   Games per season:
      22019: 1059 games
      22020: 1080 games
      22021: 1230 games
      22022: 1230 games
      22023: 1230 games
      22024: 1225 games

🔍 Checking team matchups...
   ✅ No games where team plays itself

🔍 VALIDATING P